In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...

In [3]:
import pymrio # /!\ Clone and install pymrio from https://github.com/bixiou/pymrio
from pymrio.core.mriosystem import IOSystem as IOS
from pymrio.core.mriosystem import concate_extension
from pymrio.tools.iomath import div0
from pymrio.tools.iofunctions import *
from pymrio.tools.ioparser import *
import pandas as pd
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...
warnings.filterwarnings('ignore')

In [4]:
path_io = '/mnt/dd/adrien/DD/Économie/Données/' # where the Exiobase data is

In [5]:
exio3 = pickle.load(open(path_io+'Exiobase 3.4/exio3.pkl', 'rb')) # cf. below to generate the data (from exiobase3.4_iot_2011_pxp)

In [6]:
EU27 = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK', 'IE']

steel = ['Basic iron and steel and of ferro-alloys and first products thereof', 'Secondary steel for treatment, Re-processing of secondary steel into new steel'] 
aluminum = ['Aluminium and aluminium products',  'Secondary aluminium for treatment, Re-processing of secondary aluminium into new aluminium']
cement = ['Cement, lime and plaster']
power = ['Electricity by coal',  'Electricity by gas',  'Electricity by nuclear',  'Electricity by hydro',  'Electricity by wind',
 'Electricity by petroleum and other oil derivatives',  'Electricity by biomass and waste',  'Electricity by solar photovoltaic',
 'Electricity by solar thermal',  'Electricity by tide, wave, ocean',  'Electricity by Geothermal',  'Electricity nec']
chemicals = ['Chemical and fertilizer minerals, salt and other mining and quarrying products n.e.c.', 'Chemicals nec']
# 'Precious metals',
#  'Secondary preciuos metals for treatment, Re-processing of secondary preciuos metals into new preciuos metals',
#  'Aluminium and aluminium products',
#  'Secondary aluminium for treatment, Re-processing of secondary aluminium into new aluminium',
#  'Lead, zinc and tin and products thereof',
#  'Secondary lead for treatment, Re-processing of secondary lead into new lead',
#  'Copper products',
#  'Secondary copper for treatment, Re-processing of secondary copper into new copper',
#  'Other non-ferrous metal products',
#  'Secondary other non-ferrous metals for treatment, Re-processing of secondary other non-ferrous metals into new other non-ferrous metals',
#  'Foundry work services',
#  'Fabricated metal products, except machinery and equipment (28)',
#  'Machinery and equipment n.e.c. (29)',
#  'Office machinery and computers (30)',
#  'Electrical machinery and apparatus n.e.c. (31)',
#  'Radio, television and communication equipment and apparatus (32)',
#  'Medical, precision and optical instruments, watches and clocks (33)',
#  'Motor vehicles, trailers and semi-trailers (34)',
#  'Other transport equipment (35)'


In [12]:
exio3.embodied_impact_imports('CO2 (kg)', steel, EU27) / 1e9 # in MtCO2

55.77436023656755

In [13]:
exio3.embodied_impact_imports('CO2 (kg)', power, EU27) / 1e9 # in MtCO2

0.0

In [14]:
exio3.embodied_impact_imports('CO2 (kg)', cement, EU27) / 1e9 # in MtCO2

7.75427192302997

In [15]:
exio3.embodied_impact_imports('CO2 (kg)', chemicals, EU27) / 1e9 # in MtCO2

141.1210982452209

In [16]:
exio3.embodied_impact_imports('CO2 (kg)', aluminum, EU27) / 1e9 # in MtCO2

12.633704888723567

In [17]:
exio3.embodied_impact_imports('CO2 (kg)', exio3.sectors, EU27) / 1e9 # in MtCO2

1527.6481685135705

### Prepare the data 

In [ ]:
# TODO: change directly the parser

In [ ]:
exio3 = parse_exiobase3(path_io+'Exiobase 3.4/exiobase3.4_iot_2011_pxp.zip')
exio3.calc_all()
exio3.x = exio3.x.squeeze()
exio3.impact = exio3.satellite

In [10]:
sectors = ['food', 'tobacco', 'cloth', 'housing', 'furniture', 'health', 'transport', 'communication', 'leisure', 'education', 'catering', 'diverse']
imp = exio3.impact.F.index

In [8]:
impacts = ['CO2 (kg)', 'CH4 (kg)', 'NOx (kg)', 'SOx (kg)', 'N2O (kg)', 'NH3 (kg)', 'CO (kg)', 'PFC (kg CO2eq)', 'HFC (kg CO2eq)', 'SF6 (kg)',
         'PM10 (kg)', 'PM2.5 (kg)', 'Pb (kg)', 'Cd (kg)', 'Gravel and sand (kg)',
         'Arsenic (kg)', 'NMVOC (kg)', 'TSP (kg)', 'N (kg)', 'P (kg)', 'Energy (TJ)', 'Wood (kt)', 'Copper (kt)', 'Metal Ores (kt)', 'Non-Metallic Minerals (kt)', 
         'Crops (kt)', 'Cropland (km2)', 'Forest area (km2)', 'Permanent pastures (km2)', 'Land use (km2)', 'Water Consumption Blue (Mm3)', 
         'Water Consumption Green (Mm3)', 'Water Withdrawal Blue (Mm3)', 'Wages (incl. social contributions) (M€)', 
         'Operating surplus (investment, rents, profit) (M€)', 'Value added (wages, surplus and taxes) (M€)', 'Employment (k pers)', 'Employment hours (M.hr)']

In [11]:
agg_impacts = exio3.impact.F
exio3.impact.F = agg_impacts.append([ # TODO: énergie, déchets, minerais, bois TODO: Global Warming Potential (CO2eq)
#     pd.Series(agg_impacts.iloc[['CO2 - combustion - air' in i for i in imp],:].sum(axis = 0), name = 'CO2 - combustion - air (kg)'),
    pd.Series(agg_impacts.iloc[['CO2' in i for i in imp],:].sum(axis = 0), name = 'CO2 (kg)'),
    pd.Series(agg_impacts.iloc[['CH4' in i for i in imp],:].sum(axis = 0), name = 'CH4 (kg)'),
    pd.Series(agg_impacts.iloc[['NOx' in i for i in imp],:].sum(axis = 0), name = 'NOx (kg)'),
    pd.Series(agg_impacts.iloc[['SOx' in i for i in imp],:].sum(axis = 0), name = 'SOx (kg)'),
    pd.Series(agg_impacts.iloc[['N2O' in i for i in imp],:].sum(axis = 0), name = 'N2O (kg)'),
    pd.Series(agg_impacts.iloc[['NH3' in i for i in imp],:].sum(axis = 0), name = 'NH3 (kg)'),
    pd.Series(agg_impacts.iloc[['CO -' in i for i in imp],:].sum(axis = 0), name = 'CO (kg)'),
    pd.Series(agg_impacts.iloc[['PFC' in i for i in imp],:].sum(axis = 0), name = 'PFC (kg CO2eq)'),
    pd.Series(agg_impacts.iloc[['HFC' in i for i in imp],:].sum(axis = 0), name = 'HFC (kg CO2eq)'),
    pd.Series(agg_impacts.iloc[['SF6' in i for i in imp],:].sum(axis = 0), name = 'SF6 (kg)'),
    pd.Series(agg_impacts.iloc[['PM10' in i for i in imp],:].sum(axis = 0), name = 'PM10 (kg)'),
    pd.Series(agg_impacts.iloc[['PM2.5' in i for i in imp],:].sum(axis = 0), name = 'PM2.5 (kg)'),
    pd.Series(agg_impacts.iloc[['Pb' in i for i in imp],:].sum(axis = 0), name = 'Pb (kg)'),
    pd.Series(agg_impacts.iloc[['Cd' in i for i in imp],:].sum(axis = 0), name = 'Cd (kg)'),
    pd.Series(agg_impacts.iloc[['sand' in i for i in imp],:].sum(axis = 0), name = 'Gravel and sand (kg)'),
    pd.Series(agg_impacts.iloc[['As -' in i for i in imp],:].sum(axis = 0), name = 'Arsenic (kg)'),
    pd.Series(agg_impacts.iloc[['NMVOC' in i for i in imp],:].sum(axis = 0), name = 'NMVOC (kg)'),
    pd.Series(agg_impacts.iloc[['TSP' in i for i in imp],:].sum(axis = 0), name = 'TSP (kg)'),
    pd.Series(agg_impacts.iloc[['N -' in i for i in imp],:].sum(axis = 0), name = 'N (kg)'),
    pd.Series(agg_impacts.iloc[['P - a' in i or 'P - w' in i for i in imp],:].sum(axis = 0), name = 'P (kg)'),
    pd.Series(agg_impacts.iloc[['Energy Carrier Supply' in i for i in imp],:].sum(axis = 0), name = 'Energy (TJ)'),
    pd.Series(agg_impacts.iloc[['s wood -' in i for i in imp],:].sum(axis = 0), name = 'Wood (kt)'), # from Wood to Crops, includes both used and unused extraction
    pd.Series(agg_impacts.iloc[['Ores - Copper' in i for i in imp],:].sum(axis = 0), name = 'Copper (kt)'),
    pd.Series(agg_impacts.iloc[['Metal Ores' in i for i in imp],:].sum(axis = 0), name = 'Metal Ores (kt)'),
    pd.Series(agg_impacts.iloc[['Non-Metallic Minerals' in i for i in imp],:].sum(axis = 0), name = 'Non-Metallic Minerals (kt)'),
    pd.Series(agg_impacts.iloc[['Primary Crops' in i for i in imp],:].sum(axis = 0), name = 'Crops (kt)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i for i in imp],:].sum(axis = 0), name = 'Cropland (km2)'),
    pd.Series(agg_impacts.iloc[['Forest area' in i for i in imp],:].sum(axis = 0), name = 'Forest area (km2)'),
    pd.Series(agg_impacts.iloc[['Permanent pasture' in i for i in imp],:].sum(axis = 0), name = 'Permanent pastures (km2)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i or 'land Use' in i or 'pasture' in i for i in imp],:].sum(axis = 0), name = 'Land use (km2)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Blue' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Green' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Green (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Withdrawal' in i for i in imp],:].sum(axis = 0), name = 'Water Withdrawal Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['wages' in i for i in imp],:].sum(0), name='Wages (incl. social contributions) (M€)'),
    pd.Series(agg_impacts.iloc[['surplus' in i for i in imp],:].sum(0), name='Operating surplus (investment, rents, profit) (M€)'),
    pd.Series(agg_impacts.iloc[['axes' in i or 'wages' in i or 'surplus' in i for i in imp],:].sum(0), name='Value added (wages, surplus and taxes) (M€)'),
    pd.Series(agg_impacts.iloc[['Employment:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name='Employment (k pers)'),
    pd.Series(agg_impacts.iloc[['Employment hours:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name = 'Employment hours (M.hr)')])
del(agg_impacts)

In [18]:
pickle.dump(exio3, open(path_io + 'Exiobase 3.4/exio3.pkl', 'wb'))